In [1]:
#create a buisness std data frame to concate with attributes and hours on a normalized json
#the normalized json will then dissect the dictionary into its emperical form
import pandas as pd
import numpy as np
business = pd.read_json('yelp_academic_dataset_business.json', lines =True)
business_attributes = pd.json_normalize(business['attributes'])
business_hours = pd.json_normalize(business['hours'])


In [3]:
#use result as a dummy for a sequential conocating
#keep buisness on the left for formating purposes
#axis = 1 because we dont need another index
result = pd.concat([business, business_attributes], axis=1,)

In [4]:
#see above
business = pd.concat([result, business_hours], axis =1)

In [5]:
#refer to project one on explanation of this code
#it drops the cateogries and split like a .explode function except it creates duplicates by expanding whenever , is present. 
#restack
#reset index and rename it after adding it in
business_data = business.drop('categories', axis=1).join(business['categories'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('establishment_type'))

In [6]:
#set this as type str for regex prep
business_data['establishment_type'] = business_data['establishment_type'].astype('str')

In [7]:
#acknowledge resturants is # 1 
business_data.establishment_type.value_counts(10)

 Restaurants            0.055299
 Food                   0.031401
 Shopping               0.028286
Restaurants             0.022865
 Home Services          0.015796
                          ...   
 Patent Law             0.000001
LAN Centers             0.000001
Holistic Animal Care    0.000001
Bocce Ball              0.000001
 Trade Fairs            0.000001
Name: establishment_type, Length: 2455, dtype: float64

In [8]:
#import re, find the data that only contains resturants and food with no case sensitivity 
import re 
business_dataprep = business_data[business_data['establishment_type'].str.contains('.*Restaurants.*')| business_data['establishment_type'].str.contains('.*Food.*')]

In [9]:
#drop duplicate ids because its the same resturant 
buisness_data_prep = business_dataprep.drop_duplicates(subset=['business_id'], ignore_index= True)

In [14]:
#obtain the data from philly only
geo_example_philly= buisness_data_prep_copy[buisness_data_prep_copy['city'] == 'Philadelphia']

In [16]:
#now we incorporate NLTK text merge 
import json
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.width', 10000)

# open input file: 
ifile = open('yelp_academic_dataset_review-001.json', encoding = 'ascii', errors = 'ignore') 

# read the first 100k entries
# set to -1 to process everything
stop = 600000

all_data = list()
for i, line in enumerate(ifile):
    if i%10000==0:
        print(i)
    if i==stop:
        break    
    # convert the json on this line to a dict
    data = json.loads(line)
    # extract what we want
    text = data['text']
    stars = data['stars']
    user_id = data['user_id']
    business_id= data["business_id"]
    # add to the data collected so far
    all_data.append([stars, text, user_id, business_id])
# create the DataFrame
yelp_reviews_raw = pd.DataFrame(all_data, columns=['stars','text', 'user_id', 'business_id'])

# df.to_hdf('

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000


In [19]:
yelp_philly_merge= pd.merge(geo_example_philly, yelp_reviews_raw, on='business_id', how ="left")

In [23]:
yelp_philly_merge.to_csv('EDA.csv')
days = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
